# Encoder decoder Network
### This notebook depicts the implementation of a encoder decoder network for seqeunce to sequence task. 

In [3]:
from random import randint
import numpy as np 
from keras.utils import to_categorical
from keras.models import Model 
from keras.layers import Input
from keras.layers import LSTM 
from keras.layers import Dense

In [ ]:
n_features = 50+1 
# generate a sequence of random integers 
def generate_sequence(length, n_unique):
    return [randint(1, n_unique-1) for _ in range(length)]

# Prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, n_samples):
    x1, x2, y = [],[],[]
    for _ in range(n_samples):

# generate source sequence 
        source = generate_sequence(n_in, cardinality)

# define target sequence 
        target = source[:n_out]
        target.reverse()

# create padded input target sequence 
        target_in = [0] + target[:-1]
        src_encoded = to_categorical([source], num_classes=cardinality)
        tar_encoded = to_categorical([target], num_classes=cardinality)
        tar2_encoded = to_categorical([target_in], num_classes=cardinality)
# store 
        x1.append(src_encoded)
        x2.append(tar2_encoded)
        y.append(tar_encoded)
    
    return np.array(x1), np.array(x2), np.array(y)

In [ ]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

# configure problem 
n_features = 50 + 1
n_steps_in = 6 
n_steps_out = 3 

# generate a single source and target sequence 
x1, x2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)

print(x1.shape, x2.shape, y.shape)
print(f"X1={one_hot_decode(x1[0])}, X2={one_hot_decode(x2[0])}, y= {one_hot_decode(y[0])}")

In [1]:
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
    '''
    n_input: The cardiantly of the inpute sequence 
    n_output: The cardinality of the output sequence 
    n_units: The number of cells to create in the encoder and decoder models. 
    '''
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states=[state_h, state_c]

    #define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # define inference encoder 
    encoder_model = Model(encoder_inputs, encoder_states)

    # define inference decoder 
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
    # return all models
    return model, encoder_model, decoder_model

SyntaxError: incomplete input (1554821188.py, line 1)

In [ ]:
# generate target given source sequence 
def predict_sequeence(infenc, infdec, source, n_steps, cardinality):
    '''
    infenc: Encoder model used when making a prediction for a new source sequence 
    infdec: Decoder model used when making a prediction for a new source sequence 
    source: Encoded source sequence 
    n_steps: Number of time stpes in the target sequence
    cardinality: The cardinality of the output sequence, e.g. the number of features, words, or characters for each time step 
    '''
# encode
    state = infenc.predict(source)
# start of sequence input
    target_seq = np.array([0.0 for _ in range(cardinality)]).reshape(1,1,cardinality)
# collect predictions 
    output = list()
    for t in range(n_steps):
# predict next char 
        yhat, h, c = infdec.predict([target_seq] + state)
# store prediction 
        output.append(yhat[0,0,:])
# update state 
        state = [h,c]
# update target sequence 
        target_seq = yhat 
        return np.array(output)